In [51]:
import numpy as np
import matplotlib.pyplot as plt

# Carregando dataset
dataset = np.loadtxt("dataset/aerogerador.dat")
dataset

array([[  0. ,   0. ],
       [  0. ,   0. ],
       [  0. ,   0. ],
       ...,
       [ 14.1, 508. ],
       [ 14.3, 508. ],
       [ 14.4, 503. ]])

In [52]:
vel = np.expand_dims(dataset[:, 0], axis=1)  # Velocidades
Pot = np.expand_dims(dataset[:, 1], axis=1)  # Potências

vel.shape, Pot.shape

((2250, 1), (2250, 1))

In [76]:
from sklearn.metrics import r2_score

def criteria_metrics(type, y_true, y_pred, X, num_coeff):
    if type == "R2":
        return r2_score(y_true, y_pred)
    elif type == "Adj_R2":
        return 1 - (1 - r2_score(y_true, y_pred)) * ((len(y_true) - 1) / (len(y_true) - X.shape[1] - 1))
    elif type == "AIC":
        res = y_true - y_pred
        sum_res = np.sum(res**2)
        return len(y_true) * np.log(sum_res) + 2 * num_coeff

def polynomial_matrix(k, X):
    if k == 1:
        return np.column_stack((np.ones(X.shape[0]), X)) 
    else:
        polynomials = [X ** pol for pol in range(0, k + 1)]
        return np.column_stack(polynomials)
        
def train_multi_linear_regressor_grs(X_train, y_train, degrees_list, num_exec, num_iter, l_rate):
    degrees_best_models = [] 
    
    for degree in degrees_list:
        X = polynomial_matrix(degree, X_train)
        y = y_train

        coeffs_list = []
        r2s_list = []
        adj_r2s_list = []
        aics_list = []
        
        for exec in range(num_exec):
            print("Execution {} - Degree {}".format(exec, degree))
            coefficients = np.random.rand(degree + 1) * np.sqrt(2 / (degree + 1)) # Inicialização (Xavier)
            
            for iter in range(num_iter):
                actual_y_pred = np.dot(X, coefficients)
                actual_mse_error = np.mean((actual_y_pred - y) ** 2) # Mean Square Error

                new_coefficients = coefficients + l_rate * np.random.normal(loc=0, scale=0.1, size=(degree + 1))
                new_y_pred = np.dot(X, new_coefficients)
                new_mse_error = np.mean((new_y_pred - y) ** 2) # Mean Square Error

                if new_mse_error < actual_mse_error:
                    coefficients = new_coefficients

                if iter % 100 == 0:
                    print("\t Iteration {} - MSE {}".format(iter, actual_mse_error))

            y_pred = X @ coefficients

            r2 = criteria_metrics("R2", y_train, y_pred, X, coefficients.shape[0])
            adj_r2 = criteria_metrics("Adj_R2", y_train, y_pred, X, coefficients.shape[0])
            aic = criteria_metrics("AIC", y_train, y_pred, X, coefficients.shape[0])
            
            coeffs_list.append(coefficients)
            r2s_list.append(r2)
            adj_r2s_list.append(adj_r2)
            aics_list.append(aic)

        zipped_model = list(zip(r2s_list, coeffs_list, adj_r2s_list, aics_list))
        zipped_model.sort(reverse=True)
        best_models = zipped_model[:3] # Salvando as 3 melhores execuções com base no R2 para o grau atual de polinomio
        degrees_best_models.append(best_models)
        
        
    return degrees_best_models


In [ ]:
degrees_list = [i for i in range(1, 11)]
best_models = train_multi_linear_regressor_grs(X_train=vel, y_train=Pot, degrees_list=degrees_list, num_exec=3, \
                                             num_iter=200000, l_rate=0.9)

Execution 0 - Degree 1
	 Iteration 0 - MSE 94284.61320754414
	 Iteration 100 - MSE 71194.47791562346
	 Iteration 200 - MSE 57018.56452634692
	 Iteration 300 - MSE 46026.39938001323
	 Iteration 400 - MSE 34650.134813244156
	 Iteration 500 - MSE 27763.83591640584
	 Iteration 600 - MSE 20968.52325421819
	 Iteration 700 - MSE 15607.677555079677
	 Iteration 800 - MSE 14374.977298704029
	 Iteration 900 - MSE 14205.620423128403
	 Iteration 1000 - MSE 14125.910123752266
	 Iteration 1100 - MSE 14062.761275826864
	 Iteration 1200 - MSE 14006.627971684948
	 Iteration 1300 - MSE 13945.78933296827
	 Iteration 1400 - MSE 13874.769446514172
	 Iteration 1500 - MSE 13800.559086254454
	 Iteration 1600 - MSE 13749.8627913192
	 Iteration 1700 - MSE 13693.58380995752
	 Iteration 1800 - MSE 13635.045885206824
	 Iteration 1900 - MSE 13569.643714013293
	 Iteration 2000 - MSE 13505.429863696121
	 Iteration 2100 - MSE 13440.09252840104
	 Iteration 2200 - MSE 13380.812216319564
	 Iteration 2300 - MSE 13325.29358

In [ ]:
for degree, models in enumerate(best_models):
    r2s_list, coeffs_list, adj_r2s_list, aics_list = zip(*models)
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    X = polynomial_matrix(degree + 1, vel)
    y = Pot.reshape(-1, 1)

    for i in range(3):
        coefficients = coeffs_list[i]
        y_pred = X @ coefficients
        
        sorted_order = np.argsort(X[:, 1])
        X_sorted = X[sorted_order]
        y_pred_sorted = y_pred[sorted_order]
        
        axes[i].scatter(X[:, 1], y, label='Real Data')
        axes[i].plot(X_sorted[:, 1], y_pred_sorted, color='red', label='Predictions')
        axes[i].set_title("Top {} Execution - Degree {}".format(i+1, degree+1))
        axes[i].set_xlabel('vel')
        axes[i].set_ylabel('Pot')
        axes[i].text(0, 430, 'R2: {:.2f}'.format(r2s_list[i]))
        axes[i].text(0, 400, 'Adj. R2: {:.2f}'.format(adj_r2s_list[i]))
        axes[i].text(0, 370, 'AIC: {:.2f}'.format(aics_list[i]))
        axes[i].legend()

    plt.subplots_adjust(wspace=0.3)
    plt.show()